### RankNet-check-modeldata

Check the intermediate data in the network.

testset for car12, 188 records in total

model intermediate data for the last three layers:

    target       ; 1
    theta        ; 3
    rnnoutput    ; 40


In [6]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import indycar.model.stint_simulator_shortterm_pitmodel as stint

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [145]:
### load test_ds and model saved data
prediction_length = 2
freq = "1T"
cardinality = [0]

def load_dataset(inputfile):
    global freq, prediction_length, cardinality

    with open(inputfile, 'rb') as f:
        # have to specify it.
        freq, prediction_length, cardinality,train_ds, test_ds = pickle.load(f, encoding='latin1')
    
    print(f"number of cars: {cardinality}")
    
    return train_ds, test_ds

def load_modeldata(inputfile):
    with open(inputfile, 'rb') as f:
        # have to specify it.
        savedata = pickle.load(f, encoding='latin1')
    
    #print(f"columns of data: {savedata.keys()}")
    print(f"size of data: {len(savedata)}")
    
    return savedata


In [80]:
testfile = 'testset-car12-f1min-t2-rIndy500-2018-gluonts-indy-2018.pickle'
#100 samples
#modelfile = 'ranknet_savedata_multilayers.pickle'
#1 sample
modelfile = 'ranknet_savedata_multilayers_b1_s1.pickle'


In [81]:
_, test_ds = load_dataset(testfile)
savedata = load_modeldata(modelfile)

number of cars: [58]
columns of data: dict_keys(['input', 'target', 'lags', 'theta', 'hstate', 'rnnoutput'])


In [21]:
### iter through the test set
testset = list(test_ds)
print('testset predict laps start:', testset[-1]['target'].shape,
     'end:', testset[0]['target'].shape
     )

testset predict laps start: (13,) end: (200,)


In [25]:
# lap id start from 0
pitstops = np.where(testset[0]['feat_dynamic_real'][1]==1)
pitstops

(array([ 31,  49,  93, 128, 170]),)

In [82]:
### go through the model intermediate data
len(savedata['target'])

188

In [ ]:
savedata['target'][-1].shape

In [91]:
target = np.array([x.reshape(-1)[0] for x in savedata['target']])

In [93]:
target

array([-0.44145054,  1.5119362 ,  1.0326842 ,  2.927995  ,  3.0142956 ,
        3.967255  ,  4.0858655 ,  4.033666  ,  4.2060766 ,  3.8572142 ,
        3.8496976 ,  4.01355   ,  3.8975718 ,  4.279505  ,  5.468411  ,
        4.1787167 , -6.724894  ,  5.610863  ,  5.424944  ,  6.261024  ,
        4.6065817 ,  5.95868   ,  7.468332  ,  3.2264085 ,  5.393872  ,
        6.082346  ,  7.0473547 ,  4.6004095 ,  0.6277979 ,  3.5979862 ,
        1.0074111 ,  1.0023022 ,  0.90134317,  0.78475326,  0.69376236,
        0.994295  ,  1.0061753 ,  1.0110877 ,  1.0533919 ,  0.92796665,
        0.9562135 ,  0.99053484,  0.60095453,  1.1205821 ,  1.0505611 ,
        1.166578  ,  0.1438639 ,  2.4977386 ,  1.2265257 ,  0.97292167,
        0.9915894 ,  0.9755446 ,  0.98881054,  0.9777511 ,  0.9292344 ,
        0.99056536,  0.985283  ,  1.0932226 ,  2.0129857 ,  2.0398622 ,
        2.0394232 ,  3.0486848 ,  4.0562363 ,  4.0398755 ,  4.74736   ,
        5.658873  ,  5.087961  ,  3.6991258 , 11.792514  , 11.92

In [94]:
len(savedata['input'])

188

In [95]:
savedata['input'][0].shape

(1, 40, 33)

In [96]:
len(savedata['theta'][0])

3

In [113]:
theta = np.array([[x.asnumpy().reshape(-1)[0] for x in savedata['theta'][id]] for id in range(len(savedata['theta']))])
theta

array([[ 2.11339802e-01,  8.36189538e-02,  2.01433063e+00],
       [ 3.04522395e-01,  6.54837042e-02,  2.01155996e+00],
       [ 2.69080788e-01,  5.64465076e-02,  2.00873518e+00],
       [ 7.14797914e-01,  1.93236656e-02,  2.00145078e+00],
       [ 7.16342509e-01,  1.77412890e-02,  2.00108171e+00],
       [ 9.65214550e-01,  1.53534384e-02,  2.00046992e+00],
       [ 9.99411941e-01,  1.20753273e-02,  2.00037050e+00],
       [ 1.00660443e+00,  6.87769474e-03,  2.00011659e+00],
       [ 1.02198780e+00,  3.84728587e-03,  2.00003433e+00],
       [ 1.02356279e+00,  4.56581404e-03,  2.00006270e+00],
       [ 1.01667428e+00,  6.77775638e-03,  2.00017738e+00],
       [ 1.09427059e+00,  1.07547492e-02,  2.00023317e+00],
       [ 1.08035624e+00,  1.49546228e-02,  2.00057006e+00],
       [ 1.22744310e+00,  2.68170051e-02,  2.00146604e+00],
       [ 1.53181064e+00,  4.94227819e-02,  2.00278783e+00],
       [ 1.28005981e+00,  5.65492921e-02,  2.00535607e+00],
       [-1.90894353e+00,  7.49290049e-01

In [107]:
savedata['theta'][-1][0].asnumpy().reshape(-1)[0]

1.065379

In [114]:
savedata['rnnoutput'][0].shape

(1, 1, 40)

In [119]:
rnnoutput = np.array([x.reshape(-1) for x in savedata['rnnoutput']])
rnnoutput.shape

(188, 40)

In [120]:
rnnoutput[0]

array([ 0.37186566, -0.2355668 ,  0.03524104,  0.31859934,  0.10533482,
        0.01034961,  0.1531219 , -0.06566571, -0.17070529,  0.06987295,
       -0.2244168 ,  0.03112466, -0.0458408 , -0.23805694, -0.09133456,
       -0.08574518, -0.06131275, -0.03852127,  0.2119589 , -0.29207027,
        0.0055407 ,  0.04884779, -0.3250793 ,  0.00214675, -0.04041871,
       -0.22301802, -0.09570426,  0.02288404, -0.07174075, -0.221968  ,
        0.08084089, -0.33736727,  0.33693063, -0.20730151,  0.44099346,
        0.30486134,  0.13770637, -0.06217545, -0.07174944, -0.28200722],
      dtype=float32)

In [126]:
### final 3 layers
### target, theta, rnnoutput

#id=0 -> lap 200
#id=  -> lap 94, pit stop
id = 200-94

#it is in the center of uncertainty
target[id-2:id+3]

array([ 0.07159143, -4.636854  , -7.757321  ,  1.8675687 ,  2.1143522 ],
      dtype=float32)

In [123]:
theta[id]

array([0.53898644, 1.6165512 , 7.0077815 ], dtype=float32)

In [124]:
rnnoutput[id]

array([ 0.3853559 , -0.42747563,  0.49426314,  0.28006268,  0.41069704,
       -0.02163856, -0.12062053, -0.1193699 , -0.12209579, -0.05988751,
       -0.348947  ,  0.07152105,  0.64503443, -0.16567425,  0.36619002,
       -0.1102424 ,  0.07747326,  0.06775997, -0.05031077, -0.2942695 ,
        0.04359036,  0.00119089, -0.31879818,  0.24713482,  0.22637439,
       -0.19594297, -0.00248465,  0.70728713, -0.20905301, -0.24483114,
        0.07092818, -0.49214923,  0.32881126, -0.481456  ,  0.3418061 ,
        0.06407523,  0.3439141 , -0.07951942,  0.10768818,  0.2573191 ],
      dtype=float32)

In [133]:
### check RankNet
modelfile = 'RankNet-MLP_savedata_multilayers_b1_s1_t0.pickle'
ranknet = load_modeldata(modelfile)

target = np.array([x.reshape(-1)[0] for x in ranknet['target']])
id = 200-94

#it is in the center of uncertainty
target[id-2:id+3]

columns of data: dict_keys(['input', 'target', 'lags', 'theta', 'hstate', 'rnnoutput'])


array([1.078691  , 1.0311098 , 0.9845628 , 0.98414487, 0.98873556],
      dtype=float32)

In [134]:
np.where(target<0)

(array([ 20,  22,  79,  81,  82,  83, 101, 114, 117, 140, 164]),)

In [130]:
target_o = np.array([x.reshape(-1)[0] for x in savedata['target']])

np.where(target_o<0)

(array([  0,  16,  71, 105, 106]),)

In [137]:
modelfile = 'RankNet-MLP_savedata_multilayers_b1_s1_t1.pickle'
ranknet1 = load_modeldata(modelfile)

target1 = np.array([x.reshape(-1)[0] for x in ranknet1['target']])
np.where(target1<0)

columns of data: dict_keys(['input', 'target', 'lags', 'theta', 'hstate', 'rnnoutput'])


(array([ 19,  20,  22, 112, 116, 117, 140, 141, 187]),)

In [138]:
rnnoutput = np.array([x.reshape(-1) for x in ranknet['rnnoutput']])
rnnoutput.shape

(188, 40)

In [140]:
theta = np.array([[x.asnumpy().reshape(-1)[0] for x in savedata['theta'][id]] for id in range(len(ranknet['theta']))])
theta.shape

(188, 3)

### 100 samples, test on the average model data

In [155]:
modelfile = 'RankNet-MLP_savedata_multilayers_b1_s100_t0.pickle'
ranknet = load_modeldata(modelfile)

size of data: 100


In [156]:
len(ranknet)

100

In [157]:
ranknet[0].keys()

dict_keys(['input', 'target', 'lags', 'theta', 'hstate', 'rnnoutput'])

In [158]:
np.array([x.reshape(-1) for x in ranknet[0]['rnnoutput']])[0]

array([ 0.56904113, -0.6019372 , -0.26216808,  0.5077025 ,  0.12048921,
        0.08039901,  0.12102704, -0.19108155, -0.39448035,  0.01646979,
       -0.23147646,  0.07274219, -0.27615643, -0.51665825,  0.01381567,
       -0.22633345, -0.09604313,  0.15158312,  0.20746446, -0.2075398 ,
       -0.07896389,  0.02502082, -0.15783079, -0.16771129,  0.28573942,
        0.08723599, -0.02885644, -0.4371242 ,  0.20509085, -0.39072007,
        0.33336955, -0.20196714,  0.3357562 , -0.22202078,  0.558727  ,
        0.7034129 ,  0.1746498 , -0.02207406,  0.05980825, -0.00549192],
      dtype=float32)

In [159]:
np.array([x.reshape(-1) for x in ranknet[1]['rnnoutput']])[0]

array([ 0.5690333 , -0.6019297 , -0.26216802,  0.50770587,  0.12048061,
        0.08040307,  0.12104224, -0.19107008, -0.39446583,  0.01648019,
       -0.2314671 ,  0.0727394 , -0.27615148, -0.5166325 ,  0.01378983,
       -0.22630051, -0.09605829,  0.15157767,  0.2074795 , -0.20756161,
       -0.07895461,  0.02502225, -0.15784803, -0.16768648,  0.2857059 ,
        0.08719096, -0.02885197, -0.43714333,  0.20507383, -0.39070898,
        0.33334357, -0.20197152,  0.33576122, -0.22203383,  0.5587191 ,
        0.7033926 ,  0.1746312 , -0.02204334,  0.05979649, -0.00550865],
      dtype=float32)

In [160]:

datax = ranknet[0] 

theta = np.array([[x.asnumpy().reshape(-1)[0] for x in datax['theta'][id]] for id in range(len(datax['theta']))])
theta.shape
rnnoutput = np.array([x.reshape(-1) for x in datax['rnnoutput']])
rnnoutput.shape

all_theta = np.zeros((len(ranknet), theta.shape[0], theta.shape[1]))
all_rnnoutput = np.zeros((len(ranknet), rnnoutput.shape[0], rnnoutput.shape[1]))

for id, datax in enumerate(ranknet):
    theta = np.array([[x.asnumpy().reshape(-1)[0] for x in datax['theta'][id]] for id in range(len(datax['theta']))])
    all_theta[id] = theta
    
    rnnoutput = np.array([x.reshape(-1) for x in datax['rnnoutput']])
    all_rnnoutput[id] = rnnoutput

#get average
theta =np.mean(all_theta,axis=0)
rnnoutput =np.mean(all_rnnoutput,axis=0)



In [161]:
rnnoutput[0]

array([ 0.56909108, -0.6019855 , -0.26216841,  0.50768122,  0.12054382,
        0.08037323,  0.12093048, -0.19115473, -0.39457212,  0.01640366,
       -0.2315362 ,  0.07275982, -0.27618817, -0.5168222 ,  0.01397981,
       -0.22654288, -0.09594673,  0.15161772,  0.20736882, -0.207401  ,
       -0.07902292,  0.0250117 , -0.15772112, -0.16786905,  0.28595243,
        0.08752265, -0.02888487, -0.43700207,  0.20519894, -0.39079029,
        0.333535  , -0.20193943,  0.33572411, -0.2219379 ,  0.55877734,
        0.70354207,  0.17476839, -0.02226995,  0.05988316, -0.00538551])

In [162]:
rnnoutput[-1]

array([ 0.37793113, -0.39036793,  0.14719297,  0.22188395,  0.17394874,
        0.04322035,  0.0386532 , -0.11205912, -0.19484319,  0.04279242,
       -0.22100024,  0.02399967, -0.02248291, -0.28822577, -0.08447922,
       -0.06851802, -0.09908401,  0.01193149,  0.12491202, -0.18044853,
       -0.0974895 ,  0.04102266, -0.34642028,  0.06488668, -0.18015351,
       -0.1905333 , -0.06151238,  0.32847175, -0.06691681, -0.18921025,
        0.12492408, -0.20279896,  0.20427127, -0.05627926,  0.34672294,
        0.18773277,  0.10734997, -0.13083964, -0.02075649, -0.09673294])

In [163]:
rnnoutput.shape

(188, 40)

### model data of pitmodel

In [3]:
import pickle
savefile = 'PitModel-savedata_dsel-e500-l10-10-5-student-d0.1.pickle'
with open(savefile, 'rb') as f: 
    alldata = pickle.load(f, encoding='latin1') 

In [4]:
print('alldata is a dict, model data for each car:', alldata.keys())

alldata is a dict, model data for each car: dict_keys([1, 3, 4, 6, 7, 9, 10, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 32, 33, 59, 60, 64, 66, 88, 98])


In [7]:
theta = np.array([[x.asnumpy().reshape(-1)[0] for x in alldata[12]['theta'][id]] 
                  for id in range(len(alldata[12]['theta']))])

print(len(theta))

theta

170


array([[ 1.23615313e+00,  2.54198104e-01,  3.67092848e+00],
       [ 1.13235927e+00,  2.43951127e-01,  3.74481869e+00],
       [ 1.02856517e+00,  2.34070629e-01,  3.81988335e+00],
       [ 9.24771309e-01,  2.24546850e-01,  3.89605451e+00],
       [ 8.20977449e-01,  2.15370238e-01,  3.97326517e+00],
       [ 7.17183471e-01,  2.06531227e-01,  4.05145073e+00],
       [ 6.36848867e-01,  2.09689304e-01,  4.08367348e+00],
       [ 5.49282670e-01,  2.13657677e-01,  4.15266800e+00],
       [ 4.54483688e-01,  2.13643044e-01,  4.25413370e+00],
       [ 3.59684885e-01,  2.13628381e-01,  4.35682201e+00],
       [ 2.64885902e-01,  2.13613704e-01,  4.46062660e+00],
       [ 1.70086935e-01,  2.13599026e-01,  4.56544828e+00],
       [ 7.52880424e-02,  2.13584349e-01,  4.67119360e+00],
       [-1.95109248e-02,  2.13569671e-01,  4.77777767e+00],
       [-1.10752173e-01,  2.13712960e-01,  4.87988472e+00],
       [-2.03185767e-01,  2.17365474e-01,  4.98375034e+00],
       [-2.99104959e-01,  2.29087815e-01

In [32]:
theta[40]

array([0.4544837 , 0.21364304, 4.2541337 ], dtype=float32)

In [30]:
theta[22]

array([-0.9056088 ,  0.21931472,  6.162998  ], dtype=float32)

In [8]:
mlpoutput = alldata[12]['mlpoutput']
print(len(mlpoutput))

mlpoutput[0]

170


array([[[-0.6255078 , -0.68457186, -0.46825197, -0.518488  ,
         -0.9469865 ]]], dtype=float32)

In [11]:
print('row id is the lap number')

row id is the lap number


In [10]:
df_test = pd.read_csv('PitModel-savedata_testset.csv')
df_test[df_test['carno']==12]

,Unnamed: 0,lap2nextpit,caution_laps,pitage,pit_oncaution,carno,eid,lap,stint_len
1028,1028,31.0,0.0,1.0,0.0,12,5,0,31.0
1029,1029,30.0,0.0,2.0,0.0,12,5,1,31.0
1030,1030,29.0,0.0,3.0,0.0,12,5,2,31.0
1031,1031,28.0,0.0,4.0,0.0,12,5,3,31.0
1032,1032,27.0,0.0,5.0,0.0,12,5,4,31.0
...,...,...,...,...,...,...,...,...,...
1193,1193,5.0,20.0,37.0,0.0,12,5,165,42.0
1194,1194,4.0,20.0,38.0,0.0,12,5,166,42.0
1195,1195,3.0,20.0,39.0,0.0,12,5,167,42.0
1196,1196,2.0,20.0,40.0,0.0,12,5,168,42.0


In [15]:
df12 = df_test[df_test['carno']==12]
df12[df12['lap']==30]

,Unnamed: 0,lap2nextpit,caution_laps,pitage,pit_oncaution,carno,eid,lap,stint_len
1058,1058,1.0,0.0,31.0,0.0,12,5,30,31.0


In [16]:
df12[df12['lap2nextpit']==2]

,Unnamed: 0,lap2nextpit,caution_laps,pitage,pit_oncaution,carno,eid,lap,stint_len
1057,1057,2.0,0.0,30.0,0.0,12,5,29,31.0
1075,1075,2.0,1.0,16.0,1.0,12,5,47,18.0
1119,1119,2.0,14.0,42.0,0.0,12,5,91,44.0
1154,1154,2.0,0.0,33.0,0.0,12,5,126,35.0
1196,1196,2.0,20.0,40.0,0.0,12,5,168,42.0


In [25]:
df_test[(df_test['carno']==14) & (df_test['lap2nextpit']==10)]

,Unnamed: 0,lap2nextpit,caution_laps,pitage,pit_oncaution,carno,eid,lap,stint_len
1245,1245,10.0,0.0,20.0,0.0,14,5,19,29.0
1265,1265,10.0,0.0,10.0,1.0,14,5,39,20.0
1305,1305,10.0,14.0,30.0,0.0,14,5,79,40.0
1345,1345,10.0,0.0,22.0,0.0,14,5,119,32.0
1358,1358,10.0,0.0,3.0,1.0,14,5,132,13.0


In [28]:
df_test[(df_test['carno']==7) & (df_test['lap2nextpit']==10)]

,Unnamed: 0,lap2nextpit,caution_laps,pitage,pit_oncaution,carno,eid,lap,stint_len
682,682,10.0,0.0,26.0,0.0,7,5,25,35.0
699,699,10.0,0.0,7.0,1.0,7,5,42,17.0
735,735,10.0,7.0,19.0,0.0,7,5,78,29.0
753,753,10.0,0.0,1.0,0.0,7,5,96,11.0
766,766,10.0,0.0,3.0,0.0,7,5,109,13.0
784,784,10.0,0.0,8.0,1.0,7,5,127,18.0
826,826,10.0,15.0,32.0,0.0,7,5,169,42.0


In [24]:
df_test[(df_test['carno']==12) & (df_test['lap2nextpit']==10)]

,Unnamed: 0,lap2nextpit,caution_laps,pitage,pit_oncaution,carno,eid,lap,stint_len
1049,1049,10.0,0.0,22.0,0.0,12,5,21,31.0
1067,1067,10.0,0.0,8.0,1.0,12,5,39,18.0
1111,1111,10.0,14.0,34.0,0.0,12,5,83,44.0
1146,1146,10.0,0.0,25.0,0.0,12,5,118,35.0
1188,1188,10.0,20.0,32.0,0.0,12,5,160,42.0


In [31]:
df_test[(df_test['carno']==12) & (df_test['lap2nextpit']==9)]

,Unnamed: 0,lap2nextpit,caution_laps,pitage,pit_oncaution,carno,eid,lap,stint_len
1050,1050,9.0,0.0,23.0,0.0,12,5,22,31.0
1068,1068,9.0,0.0,9.0,1.0,12,5,40,18.0
1112,1112,9.0,14.0,35.0,0.0,12,5,84,44.0
1147,1147,9.0,0.0,26.0,0.0,12,5,119,35.0
1189,1189,9.0,20.0,33.0,0.0,12,5,161,42.0


In [26]:
df_test[(df_test['caution_laps']==0) & (df_test['pitage']==10)]

,Unnamed: 0,lap2nextpit,caution_laps,pitage,pit_oncaution,carno,eid,lap,stint_len
9,9,22.0,0.0,10.0,0.0,1,5,9,31.0
41,41,8.0,0.0,10.0,1.0,1,5,41,18.0
110,110,24.0,0.0,10.0,0.0,1,5,110,34.0
178,178,21.0,0.0,10.0,0.0,3,5,9,30.0
209,209,9.0,0.0,10.0,1.0,3,5,40,19.0
...,...,...,...,...,...,...,...,...,...
4741,4741,10.0,0.0,10.0,1.0,88,5,180,20.0
4760,4760,22.0,0.0,10.0,0.0,98,5,9,31.0
4792,4792,8.0,0.0,10.0,1.0,98,5,41,18.0
4852,4852,23.0,0.0,10.0,0.0,98,5,101,33.0


In [27]:
df_test[(df_test['carno']==88) &(df_test['caution_laps']==0) & (df_test['pitage']==10)]

,Unnamed: 0,lap2nextpit,caution_laps,pitage,pit_oncaution,carno,eid,lap,stint_len
4570,4570,22.0,0.0,10.0,0.0,88,5,9,31.0
4602,4602,8.0,0.0,10.0,1.0,88,5,41,18.0
4663,4663,21.0,0.0,10.0,0.0,88,5,102,31.0
4694,4694,7.0,0.0,10.0,1.0,88,5,133,17.0
4741,4741,10.0,0.0,10.0,1.0,88,5,180,20.0
